<a href="https://colab.research.google.com/github/SzymonNowakowski/Machine-Learning-2024/blob/master/Lab07_gradient-boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 7 - Gradient Boosting

### Author: Szymon Nowakowski


# Introduction
-------------------------
Boosting (and Gradient Boosting) is a powerful ensemble learning method that builds models sequentially to correct errors made by previous models. Unlike Random Forests, which construct trees independently and then aggregate their outputs, Boosting trains models in a stage-wise fashion, minimizing a loss function at each step.

Since you have already studied **CART (Classification and Regression Trees)** and **Random Forests**, this chapter introduces Gradient Boosting as an advanced tree-based method that often outperforms Random Forests in predictive tasks.

# Algorithm: AdaBoost
------------------

Before we begin, let's review the **Adaptive Boosting (AdaBoost) algorithm** for classification.

We assume a training dataset of $n$ samples, $\{(x_i, y_i)\}_{i=1}^{n}$, where $x_i \in \mathbb{R}^d$ are input features and $y_i \in \{-1, +1\}$ are binary class labels.

---

## **Algorithm: AdaBoost**
**Input**: Training data $\{(x_i, y_i)\}_{i=1}^{n}$, weak learner $h(x; \theta)$, number of iterations $M$  
**Output**: Final ensemble model $F_M(x)$  

1. **Initialize** sample weights:  
   $$ w_i^{(1)} = \frac{1}{n}, \quad \forall i \in \{1, \dots, n\} $$

2. **For** $m = 1$ to $M$ **do**:
   1. Train weak classifier $h_m(x)$ using weighted dataset $\{(x_i, y_i, w_i^{(m)})\}$.

   2. Compute weighted classification error:
      $$ \epsilon_m = \frac{\sum_{i=1}^{n} w_i^{(m)} \mathbb{1}(h_m(x_i) \neq y_i)}{\sum_{i=1}^{n} w_i^{(m)}} $$

   3. Compute model weight:
      $$ \alpha_m = \frac{1}{2} \log \frac{1 - \epsilon_m}{\epsilon_m} $$

   4. Update sample weights:
      $$ w_i^{(m+1)} = w_i^{(m)} \exp\left(-\alpha_m y_i h_m(x_i) \right) $$

   5. Normalize weights:
      $$ w_i^{(m+1)} = \frac{w_i^{(m+1)}}{\sum_{j=1}^{n} w_j^{(m+1)}} $$

3. **Return** final classification model:
   $$ F_M(x) = \sum_{m=1}^{M} \alpha_m h_m(x) $$
   $$ \hat{y}(x) = \text{sign}(F_M(x)) $$

---

## **Notes:**
- The weak learner $h(x; \theta)$ is typically a **stump** (a decision tree of depth 1).
- AdaBoost assigns **higher weights to misclassified samples**, making future weak learners focus on harder cases.
- The model weight $\alpha_m$ determines how much influence each weak classifier has on the final prediction.
- The final ensemble decision is based on the **weighted vote** of all weak classifiers.
- Unlike standard regression boosting, AdaBoost uses **exponential reweighting** instead of residual fitting.



# Algorithm: Regression Boosting
------------------

Let's also review the boosting algorithm for regression.

We assume a training dataset of $n$ samples, $\{(x_i, y_i)\}_{i=1}^{n}$, where $x_i \in \mathbb{R}^d$ are input features and $y_i \in \mathbb{R}$ are target values.




## **Algorithm: Regression Boosting**
**Input**: Training data $\{(x_i, y_i)\}_{i=1}^{n}$, weak learner $h(x; \theta)$, number of iterations $M$  
**Output**: Final ensemble model $F_M(x)$  

1. **Initialize** the ensemble model with a constant value (e.g., the mean target value):
   $$ F_0(x) = \frac{1}{n} \sum_{i=1}^{n} y_i $$

2. **For** $m = 1$ to $M$ **do**:
   1. Compute the residuals:
      $$ r_i^{(m)} = y_i - F_{m-1}(x_i), \quad \forall i \in \{1, \dots, n\} $$

   2. Fit a weak learner $h_m(x)$ to predict residuals:
      
      $$ h_m = \arg\min_{\theta} \sum_{i=1}^{n} \left( r_i^{(m)} - h(x_i; \theta) \right)^2 $$

   3. Update the ensemble model:
      $$ F_m(x) = F_{m-1}(x) + \lambda h_m(x) $$

      where $\lambda$ is a shrinkage parameter (learning rate) controlling the contribution of each weak learner.

3. **Return** final model $F_M(x)$.

---

## **Notes:**
- The weak learner $h(x; \theta)$ is typically a shallow regression tree or another simple model.
- This algorithm directly fits weak learners to residual errors.
- The shrinkage parameter $\lambda$ prevents overfitting by controlling the influence of each weak model.




## The Idea Behind Gradient Boosting

Gradient Boosting builds an additive model of weak learners (typically decision trees) to minimize a given loss function. The model is updated iteratively:

1. Start with an initial model, typically a constant value:

   $$ F_0(x) = \arg\min_c \sum_{i=1}^{n} L(y_i, c) $$

   where $L(y_i, c)$ is the loss function (e.g., squared loss for regression or log loss for classification).

2. For each iteration $m$, compute the residuals (pseudo-residuals):

   $$ r_{i}^{(m)} = -\left[ \frac{\partial L(y_i, F(x_i))}{\partial F(x_i)} \right]_{F=F_{m-1}} $$

   This represents the negative gradient of the loss function.

3. Fit a new weak learner $h_m(x)$ (a shallow decision tree) to predict these residuals.

4. Update the model:

   $$ F_m(x) = F_{m-1}(x) + \eta h_m(x) $$

   where $\eta$ (learning rate) controls the contribution of each weak learner.



## Comparing Gradient Boosting to Random Forests

| Feature              | Random Forests                          | Gradient Boosting                    |
|----------------------|--------------------------------|--------------------------------|
| Model Structure     | Ensemble of independent trees | Trees built sequentially       |
| Training Process   | Uses bagging (parallel training) | Boosting (sequential corrections) |
| Overfitting Risk   | Lower (averaging reduces variance) | Higher (but can be controlled) |
| Performance        | Strong, but may not optimize loss | Often superior for complex tasks |
| Speed             | Faster (can be parallelized) | Slower (sequential training) |





## Hyperparameters in Gradient Boosting

Tuning hyperparameters is crucial for Gradient Boosting performance. Key parameters include:

- **Learning Rate $\eta$**: Controls how much each tree contributes. Small values (e.g., 0.01) require more trees.
- **Number of Trees $M$**: Too many trees may lead to overfitting.
- **Tree Depth $d$**: Controls the complexity of each tree. Shallow trees (e.g., depth = 3-5) work well.
- **Min Samples Split & Min Samples Leaf**: Define when to stop growing trees.
- **Subsample**: Fraction of data used to train each tree, introducing randomness (like in Random Forests).



In the next section, we will implement Gradient Boosting in Python using `scikit-learn` and `XGBoost`.
